In [1]:
import pandas as pd
import glob
import numpy as np

In [11]:
## set paths
meta_file_path = "/well/doherty/users/cxx579/project_data/raine/gen1_26/gen1-sleep-metadata-jun20.csv"
label_path = "/well/doherty/users/cxx579/project_data/raine/gen1_26/sleepLabels"
epoch_path = "/well/doherty/users/cxx579/project_data/raine/gen1_26/gt3x/wrist/baseline_oct2/epoch"
linker_file_path = "/well/doherty/users/cxx579/project_data/raine/gen1_26/acc-linker.csv"

In [12]:
meta_df = pd.read_csv(meta_file_path)
linkerDf = pd.read_csv(linker_file_path, sep=r'\s*,\s*', engine='python')

In [13]:
sleep_label_desp = label_path + '/*.csv'
sleep_label_file_paths = glob.glob(sleep_label_desp)

In [14]:
test_path = sleep_label_file_paths[0]

In [15]:
def composeFileName(myDay, myMonth, myYear, myID):
    newDate = myYear + '-' + myMonth + '-' + myDay
    # full_file_name = str(myID) + '_wrist_' + newDate + '.gt3x'
    full_file_name = str(myID) + '_wrist_' + newDate + '-epoch.csv.gz'
    return full_file_name

In [16]:
def getNewID(oldID, linkerDf):
    match = linkerDf[linkerDf['oldID'] == oldID]
    if match.empty:
        return -1
    else:
        return match.iloc[0].newID

In [17]:
def sleepPath2SubjectIDAndDate(sleep_label_path, linkerDf):    
    # this subject id is the Raine id
    # example: well/doherty/users/cxx579/project_data/raine/gen1_26/sleepLabels/07112016-3_138001.csv'
    idAndDate = sleep_label_path.split('/')[-1]
    
    idAndDate = idAndDate[:-4]
    myDay = idAndDate[:2]
    myMonth = idAndDate[2:4]
    myYear = idAndDate[4:8]
    myID = idAndDate.split('_')[-1]
    
    newID = getNewID(myID, linkerDf)
    newFileName = composeFileName(myDay, myMonth, myYear, newID)
    return myID, newFileName 

In [18]:
oldID, epochFileName = sleepPath2SubjectIDAndDate(test_path, linkerDf)

## Merge the epoch data with labels

* Obtain the starting and end date of the experiment from meata 
* Read in the epoch file together with subject id and age at the time of the study
* Select the epoch in within the right range 
* Export all the files individually to a dest directory

In [2]:
import datetime
import gzip

In [143]:
meta_df

,Study ID,Gender,DOB,Study Date,Age at Study,Study Duration,TST (min),SOL (min),SE (%),WASO (min),...,PLM Index,Total Apnea & Hypopnea (include RERA),TST/60,#resp events/TST (hr)=AHI,Total Apnea & Hypopnea during sleep (exclu RERA),#resp events/TST (hr)=AHI (only during sleep,QC Codes,QC Notes,Notes,Unnamed: 131
0,301,Female,08/10/1968,24/01/2017,48,465.0,426.0,15.0,91.6,24.0,...,0.0,52.0,7.1,7.3,45.0,6.3,NaN,NaN,NaN,NaN
1,302,Male,24/06/1960,24/01/2017,56,466.5,377.0,6.0,80.8,83.5,...,0.6,138.0,6.3,22.0,127.0,20.2,NaN,NaN,NaN,NaN
2,801,Female,12/10/1959,25/11/2016,56,436.5,405.0,12.0,92.8,19.5,...,0.0,99.0,6.8,14.7,90.0,13.3,NaN,NaN,NaN,NaN
3,802,Male,12/03/1960,14/07/2015,55,512.0,225.0,6.5,43.9,229.0,...,0.0,108.0,3.8,28.8,102.0,27.2,slept less than 4 hours,NaN,slept less than 4 hours,NaN
4,1201,Female,27/05/1961,02/06/2015,54,463.0,310.5,25.5,67.1,126.5,...,0.0,3.0,5.2,0.6,1.0,0.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,272801,Female,16/01/1958,30/05/2016,58,452.5,407.5,7.0,90.1,38.0,...,0.0,70.0,6.8,10.3,70.0,10.3,NaN,NaN,NaN,NaN
1002,272902,Male,17/09/1964,11/10/2016,52,449.5,382.5,4.0,85.1,63.0,...,0.0,60.0,6.4,9.4,27.0,4.2,NaN,NaN,NaN,NaN
1003,273101,Female,28/11/1963,05/07/2016,52,445.5,358.0,37.0,80.4,46.0,...,0.0,49.0,6.0,8.2,38.0,6.4,NaN,NaN,NaN,NaN
1004,273501,Female,19/12/1962,19/05/2016,53,508.0,367.0,46.5,72.2,93.5,...,11.6,29.0,6.1,4.7,22.0,3.6,NaN,NaN,NaN,NaN


In [144]:
# 1. Obtain the starting and end date for sleep from meta
match = meta_df[meta_df['Study ID'] == int(oldID)]

In [145]:
match['Study Date']

540    07/11/2016
Name: Study Date, dtype: object

In [146]:
epochFileName

'970557_wrist_2016-11-07-epoch.csv.gz'

In [147]:
study_date = match.iloc[0]['Study Date']

In [148]:
timeDiff = datetime.timedelta(days=1)
start_date = datetime.datetime.strptime(study_date, '%d/%m/%Y')
end_date = date_time_object + timeDiff

In [149]:
print(start_date, end_date)

2016-11-07 00:00:00 2016-11-08 00:00:00


In [150]:
# 2. Read in the epoch file
epochFile2load = epoch_path + '/' + epochFileName



In [151]:
epochFile2load

'/well/doherty/users/cxx579/project_data/raine/gen1_26/gt3x/wrist/baseline_oct2/epoch/970557_wrist_2016-11-07-epoch.csv.gz'

In [2]:
epochFile2load = '/well/doherty/users/cxx579/project_data/test_raine/baseline-processed-sep30/epoch/770514_wrist_2016-01-18-epoch.csv.gz'

In [22]:
ok = pd.read_csv(epochFile2load, compression='gzip')

In [23]:
ok

,time,enmoTrunc,enmoAbs,xMean,yMean,zMean,xRange,yRange,zRange,xStd,...,vmfft9,vmfft10,vmfft11,vmfft12,temp,samples,dataErrors,clipsBeforeCalibr,clipsAfterCalibr,rawSamples
0,2016-01-18 18:00:00.033+0800 [Australia/Perth],0.032025,0.052489,0.086799,-0.024406,-0.877217,3.083986,2.763018,3.709659,0.342309,...,0.002989,0.002772,0.002179,0.001704,1.0,900,0,0,0,901
1,2016-01-18 18:00:30.033+0800 [Australia/Perth],0.048728,0.081089,0.315455,0.136959,0.159979,3.912907,2.856849,2.598013,0.733098,...,0.001671,0.001558,0.001141,0.001993,1.0,900,0,0,0,902
2,2016-01-18 18:01:00.033+0800 [Australia/Perth],0.098943,0.164298,-0.505712,-0.492273,-0.340835,1.966881,2.971642,2.457346,0.352210,...,0.000548,0.002045,0.002841,0.002865,1.0,900,0,0,0,902
3,2016-01-18 18:01:30.033+0800 [Australia/Perth],0.123757,0.188618,-0.293088,-0.720234,-0.405453,0.960691,2.902531,1.057048,0.140695,...,0.001155,0.001458,0.000633,0.000841,1.0,900,0,0,0,902
4,2016-01-18 18:02:00.033+0800 [Australia/Perth],0.113680,0.172027,-0.299752,-0.894456,-0.369467,2.074300,1.662003,2.575827,0.208295,...,0.000697,0.000859,0.000737,0.000679,1.0,900,0,0,0,902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21115,2016-01-26 01:57:30.033+0800 [Australia/Perth],0.001295,0.001853,0.024172,0.095770,-0.995833,0.032958,0.025954,0.022304,0.004250,...,0.000010,0.000003,0.000017,0.000018,1.0,900,0,0,0,902
21116,2016-01-26 01:58:00.033+0800 [Australia/Perth],0.007084,0.013079,-0.075981,0.128451,-0.961521,1.514588,0.479195,0.995604,0.205796,...,0.000540,0.000551,0.000623,0.000666,1.0,900,0,0,0,902
21117,2016-01-26 01:58:30.033+0800 [Australia/Perth],0.000707,0.001853,-0.056758,0.161726,-0.984749,0.023969,0.023693,0.014811,0.002722,...,0.000013,0.000006,0.000013,0.000022,1.0,900,0,0,0,902
21118,2016-01-26 01:59:00.033+0800 [Australia/Perth],0.000606,0.001801,-0.055445,0.163080,-0.984446,0.026965,0.023957,0.014987,0.002888,...,0.000011,0.000006,0.000013,0.000012,1.0,900,0,0,0,902


In [11]:
import re

test_str = "2016-04-12 18:00:00.033+0800 [Australia/Perth]"
tz = re.search(r'(?<=\[).+?(?=\])', test_str)


In [13]:
timeZone = tz.group()

In [5]:
def date_parser(t):
    '''
    Parse date a date string of the form e.g.
    2020-06-14 19:01:15.123+0100 [Europe/London]
    '''
    tz = re.search(r'(?<=\[).+?(?=\])', t)
    if tz is not None:
        tz = tz.group()
    t = re.sub(r'\[(.*?)\]', '', t)
    return pd.to_datetime(t, utc=True).tz_convert(tz)


In [6]:
e = pd.read_csv(
    epochFile2load, index_col=['time'],
    parse_dates=['time'], date_parser=date_parser,
)

In [7]:
e

,enmoTrunc,enmoAbs,xMean,yMean,zMean,xRange,yRange,zRange,xStd,yStd,...,vmfft9,vmfft10,vmfft11,vmfft12,temp,samples,dataErrors,clipsBeforeCalibr,clipsAfterCalibr,rawSamples
time,,,,,,,,,,,,,,,,,,,,,
2016-01-18 18:00:00.033000+08:00,0.032025,0.052489,0.086799,-0.024406,-0.877217,3.083986,2.763018,3.709659,0.342309,0.230275,...,0.002989,0.002772,0.002179,0.001704,1.0,900,0,0,0,901
2016-01-18 18:00:30.033000+08:00,0.048728,0.081089,0.315455,0.136959,0.159979,3.912907,2.856849,2.598013,0.733098,0.216277,...,0.001671,0.001558,0.001141,0.001993,1.0,900,0,0,0,902
2016-01-18 18:01:00.033000+08:00,0.098943,0.164298,-0.505712,-0.492273,-0.340835,1.966881,2.971642,2.457346,0.352210,0.507979,...,0.000548,0.002045,0.002841,0.002865,1.0,900,0,0,0,902
2016-01-18 18:01:30.033000+08:00,0.123757,0.188618,-0.293088,-0.720234,-0.405453,0.960691,2.902531,1.057048,0.140695,0.602686,...,0.001155,0.001458,0.000633,0.000841,1.0,900,0,0,0,902
2016-01-18 18:02:00.033000+08:00,0.113680,0.172027,-0.299752,-0.894456,-0.369467,2.074300,1.662003,2.575827,0.208295,0.236415,...,0.000697,0.000859,0.000737,0.000679,1.0,900,0,0,0,902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-26 01:57:30.033000+08:00,0.001295,0.001853,0.024172,0.095770,-0.995833,0.032958,0.025954,0.022304,0.004250,0.003794,...,0.000010,0.000003,0.000017,0.000018,1.0,900,0,0,0,902
2016-01-26 01:58:00.033000+08:00,0.007084,0.013079,-0.075981,0.128451,-0.961521,1.514588,0.479195,0.995604,0.205796,0.042461,...,0.000540,0.000551,0.000623,0.000666,1.0,900,0,0,0,902
2016-01-26 01:58:30.033000+08:00,0.000707,0.001853,-0.056758,0.161726,-0.984749,0.023969,0.023693,0.014811,0.002722,0.002764,...,0.000013,0.000006,0.000013,0.000022,1.0,900,0,0,0,902


In [8]:
import pytz

In [14]:
tz = pytz.timezone(timeZone)


In [15]:
    startTime = e.index.values[0]
    endTime = e.index.values[-1]

In [16]:
startTime

numpy.datetime64('2016-01-18T10:00:00.033000000')

In [22]:
myTime = e.index[0]

In [24]:
date_strftime(myTime)

'2016-01-18 18:00:00.033000+0800 [Australia/Perth]'

In [23]:
def date_strftime(t):
    '''
    Convert to time format of the form e.g.
    2020-06-14 19:01:15.123+0100 [Europe/London]
    '''
    tz = t.tz
    return t.strftime(f'%Y-%m-%d %H:%M:%S.%f%z [{tz}]')

In [31]:
print(sys.path)

['/gpfs3/well/doherty/users/cxx579/test_process/myBBAA', '/apps/eb/skylake/software/Python/3.7.2-GCCcore-8.2.0/easybuild/python', '/apps/eb/skylake/software/Anaconda3/2019.10/lib/python37.zip', '/apps/eb/skylake/software/Anaconda3/2019.10/lib/python3.7', '/apps/eb/skylake/software/Anaconda3/2019.10/lib/python3.7/lib-dynload', '', '/users/healthdatacdt-students/cxx579/.local/lib/python3.7/site-packages', '/apps/eb/skylake/software/Anaconda3/2019.10/lib/python3.7/site-packages', '/apps/eb/skylake/software/Anaconda3/2019.10/lib/python3.7/site-packages/IPython/extensions', '/gpfs3/users/healthdatacdt-students/cxx579/.ipython']
